# HOME CREDIT DEFAULT RISK


To make the notebook run as expected You'll need to authorize your pc to kaggle
For that grab your username and api token and paste them below 
Make sure to participate in competition named "Home Credit Default Risk". So that you'll be allowed to access the data. 


In [ ]:
kaggle_username = 'paste it here'
kaggle_key = 'paste key here'

You may need to change the paths structure below as per your os

In [ ]:
import os
_n = os.getcwd()
if not os.path.isdir('/root/.kaggle'):
    os.mkdir('/root/.kaggle')
os.chdir('/root/.kaggle')
file = open('kaggle.json','w')
file.write('{"username":"'+kaggle_username+'","key":"'+kaggle_key+'"}')
file.close()
os.chdir(_n)

In [7]:
!kaggle competitions download -c home-credit-default-risk
!unzip application_train.csv.zip
!unzip application_test.csv.zip
!unzip sample_submission.csv.zip

{"username":"paste it here","key":"paste key here"}


In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
dataset = pd.read_csv('application_train.csv')
X = dataset.iloc[:,2:].values
y = dataset.iloc[:,1].values

Splitting the dataset to make different kind of feature engineering on them.

In [ ]:
X1 = X[:,(0,1,2,3,9,10,11,12,13,26,30,38,84,85,87,88)]
X_dataset = pd.DataFrame(X)
X2 = X_dataset.drop([0,1,2,3,9,10,11,12,13,26,30,38,84,85,87,88],axis = 1)

All the feature engineering and training with a very simple xgboost(Cause machine learning is mostly data cleaning)

In [ ]:
X1 = pd.DataFrame(X1)
X1 = X1.fillna('njo')
X1 = X1.values
X2 = X2.values
from sklearn.preprocessing import Imputer
imputer = Imputer(axis = 1)
X2 = imputer.fit_transform(X2)
X_arranged = np.hstack((X1,X2))
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
column = 0
while (column<16):
    X_arranged[:,column] = labelencoder_X.fit_transform(X_arranged[:,column])
    column = column + 1
onehotencoder = OneHotEncoder(categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
X = onehotencoder.fit_transform(X_arranged).toarray()
only = X[0:100,:]
#feature scaling
print("feature scaling")
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

print("regression coming")
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X,y)


making the prediction and saving the submission csv file to current directory.

In [ ]:
testset = pd.read_csv('application_test.csv')
X_test = testset.iloc[:,1:].values
X_test1 = X_test[:,(0,1,2,3,9,10,11,12,13,26,30,38,84,85,87,88)]
Xtestdataset = pd.DataFrame(X_test)
X_test2 = Xtestdataset.drop([0,1,2,3,9,10,11,12,13,26,30,38,84,85,87,88],axis = 1)
X_test1 = pd.DataFrame(X_test1)
X_test1 = X_test1.fillna('njo')
X_test1 = X_test1.values
X_test2 = X_test2.values
X_test2 = imputer.transform(X_test2)
Xtestarranged = np.hstack((X_test1, X_test2))
column = 0
while (column<16):
    Xtestarranged[:,column] = labelencoder_X.fit_transform(Xtestarranged[:,column])
    column = column + 1
X_testfinal = onehotencoder.transform(Xtestarranged).toarray()
X_testfinal = sc_X.transform(X_testfinal)



In [ ]:
y_pred = classifier.predict_proba(X_testfinal)
samplesub = pd.read_csv('sample_submission.csv')
sub = samplesub.values
all1 = np.hstack((sub,y_pred))
allpd = pd.DataFrame(all1)
submitk = allpd.drop([1,2],axis = 1)
submitk.columns = ["SK_ID_CURR", "TARGET"]
submitk[['SK_ID_CURR']] = submitk[['SK_ID_CURR']].astype('int')
submitk = submitk.drop([],axis=1)
submitk.to_csv('submission_file.csv',index=False)
